<a href="https://colab.research.google.com/github/vsnupoudel/data-science-capstone-JohnsHopkins/blob/main/Next_word_Prediction_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Download the text files and the Embedding file

In [1]:
!gdown --id 1hlvOXJqe3fdxjQilNfLRwPud_KWcOVEE -O '/content/en_US.zip'

Downloading...
From: https://drive.google.com/uc?id=1hlvOXJqe3fdxjQilNfLRwPud_KWcOVEE
To: /content/en_US.zip
262MB [00:02, 95.9MB/s]


In [2]:
!unzip -o en_US.zip 

Archive:  en_US.zip
  inflating: en_US/en_US.blogs.txt   
  inflating: en_US/en_US.news.txt    
  inflating: en_US/en_US.twitter.txt  


In [3]:
 !gdown --id 1r3MHZdvnlC_BG8z6Jr1td-zJTC2GdqYK -O 'glove.6B.50d.txt' 

Downloading...
From: https://drive.google.com/uc?id=1r3MHZdvnlC_BG8z6Jr1td-zJTC2GdqYK
To: /content/glove.6B.50d.txt
171MB [00:01, 92.4MB/s]


In [4]:
# # establish connections with three files 
with open('./en_US/en_US.blogs.txt','r') as f:
  blogs= f.readlines(99999)

References :


 [Towards Data Science Article](https://towardsdatascience.com/exploring-the-next-word-predictor-5e22aeb85d8f)

[Machine Learning Mastery](https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/)

In [5]:
string = ' '.join(blogs) 
del blogs

In [6]:
from keras.preprocessing.text import Tokenizer
import nltk
from nltk.tokenize import word_tokenize
import numpy as np
import re
from keras.utils import to_categorical

In [7]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Removing non word characters

In [8]:
string = re.sub(pattern='\W+', repl=" ", string = string ).lower()

Tokenize and generate ngrams

In [9]:
tokens = word_tokenize(string)
train_len = 3
text_sequences = []

for i in range(train_len,len(tokens)):
  seq = tokens[i-train_len:i]
  text_sequences.append(seq)

Read in 50 dimensional embeddings

In [10]:
import pandas as pd
import numpy as np
colnames = ['word']+[ 'dim_'+str(i) for i in range(1,51)]
Embed50 = pd.read_csv('glove.6B.50d.txt', engine='python'
                  ,header=None, delim_whitespace=True ,
                  names = colnames)
Embed50.head(2)

,word,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,dim_9,dim_10,dim_11,dim_12,dim_13,dim_14,dim_15,dim_16,dim_17,dim_18,dim_19,dim_20,dim_21,dim_22,dim_23,dim_24,dim_25,dim_26,dim_27,dim_28,dim_29,dim_30,dim_31,dim_32,dim_33,dim_34,dim_35,dim_36,dim_37,dim_38,dim_39,dim_40,dim_41,dim_42,dim_43,dim_44,dim_45,dim_46,dim_47,dim_48,dim_49,dim_50
0,the,0.418000,0.24968,-0.41242,0.12170,0.34527,-0.044457,-0.49688,-0.17862,-0.00066,-0.65660,0.27843,-0.147670,-0.55677,0.14658,-0.00951,0.011658,0.10204,-0.127920,-0.84430,-0.12181,-0.016801,-0.33279,-0.15520,-0.23131,-0.191810,-1.8823,-0.76746,0.099051,-0.421250,-0.19526,4.0071,-0.18594,-0.522870,-0.31681,0.000592,0.007445,0.17778,-0.15897,0.012041,-0.054223,-0.298710,-0.15749,-0.34758,-0.045637,-0.44251,0.18785,0.002785,-0.18411,-0.115140,-0.78581
1,",",0.013441,0.23682,-0.16899,0.40951,0.63812,0.477090,-0.42852,-0.55641,-0.36400,-0.23938,0.13001,-0.063734,-0.39575,-0.48162,0.23291,0.090201,-0.13324,0.078639,-0.41634,-0.15428,0.100680,0.48891,0.31226,-0.12520,-0.037512,-1.5179,0.12612,-0.024420,-0.042961,-0.28351,3.5416,-0.11956,-0.014533,-0.14990,0.218640,-0.334120,-0.13872,0.31806,0.703580,0.448580,-0.080262,0.63003,0.32111,-0.467650,0.22786,0.36034,-0.378180,-0.56657,0.044691,0.30392


Tokenizer class from keras.preprocessing.text

In [11]:
del tokens
del seq
del string

Delete those rows which have words, that are not there in the pretrained embeddings. Found 100 such sequences

In [12]:
embedlist = list(Embed50.word)
delete_list = []
for seq in text_sequences:
  check =  all(item in embedlist for item in seq)
  if not check:
    delete_list.append(seq)

text_sequences_del = [x for x in text_sequences if x not in delete_list]
del text_sequences

In [13]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences_del)
sequences = tokenizer.texts_to_sequences(text_sequences_del)
#vocabulary size increased by 1 
vocabulary_size = len(tokenizer.word_counts)+1
n_sequences = np.asarray ( sequences , dtype='int32')

In [14]:
dic = pd.DataFrame( tokenizer.word_index.items(), columns= ['word','index'])

In [15]:
dic.shape

(4492, 2)

In [16]:
word_embeds_pre = pd.merge(
    left = dic,
    right = Embed50 ,
    how="inner",
    on="word" ).iloc[: , 2:52] 

In [17]:
word_embeds_pre.shape

(4492, 50)

In [18]:
# X and Y
train_inputs = n_sequences[:,:-1]
train_targets = n_sequences[:,-1]
train_targets = to_categorical(  train_targets 
                              , num_classes = vocabulary_size  )
seq_len = train_inputs.shape[1]

In [19]:
train_inputs.shape

(18343, 2)

In [20]:
train_targets.shape

(18343, 4493)

In [21]:
# Define model

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
model = Sequential()
embed_layer = model.add(Embedding( input_dim = vocabulary_size-1
                                  , output_dim = 50 
                                  , input_length = seq_len
                                  , weights = [word_embeds_pre]
                                  , trainable = False
                                   ) )
model.add(LSTM(100,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(100,activation='relu'))
model.add(Dense(vocabulary_size, activation='softmax'))

# compiling the network
model.compile(loss='categorical_crossentropy', optimizer='adam'
, metrics=['accuracy'])

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 2, 50)             224600    
_________________________________________________________________
lstm (LSTM)                  (None, 2, 100)            60400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                30200     
_________________________________________________________________
dense (Dense)                (None, 100)               5100      
_________________________________________________________________
dense_1 (Dense)              (None, 4493)              453793    
Total params: 774,093
Trainable params: 549,493
Non-trainable params: 224,600
_________________________________________________________________


In [ ]:
#fit model
model.fit(train_inputs,train_targets,epochs=555,verbose=1)

Epoch 1/555
574/574 [==============================] - 37s 6ms/step - loss: 7.3726 - accuracy: 0.0410
Epoch 2/555
574/574 [==============================] - 3s 6ms/step - loss: 6.6745 - accuracy: 0.0500
Epoch 3/555
574/574 [==============================] - 3s 6ms/step - loss: 6.6968 - accuracy: 0.0473
Epoch 4/555
574/574 [==============================] - 3s 6ms/step - loss: 6.6088 - accuracy: 0.0489
Epoch 5/555
574/574 [==============================] - 3s 6ms/step - loss: 6.4817 - accuracy: 0.0521
Epoch 6/555
574/574 [==============================] - 3s 6ms/step - loss: 6.3503 - accuracy: 0.0581
Epoch 7/555
574/574 [==============================] - 3s 6ms/step - loss: 6.1236 - accuracy: 0.0659
Epoch 8/555
574/574 [==============================] - 3s 6ms/step - loss: 5.9298 - accuracy: 0.0728
Epoch 9/555
574/574 [==============================] - 3s 6ms/step - loss: 5.7270 - accuracy: 0.0737
Epoch 10/555
574/574 [==============================] - 3s 6ms/step - loss: 5.5448 - accur

In [ ]:
train_targets.shape

Predicting:

In [ ]:
from keras.preprocessing.sequence import pad_sequences

input_text = input().strip().lower()
encoded_text = tokenizer.texts_to_sequences([input_text])[0]
pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')
print('input: ', input_text, '\nencoded text: ', encoded_text
      , '\n pad encoded: ' , pad_encoded)

In [ ]:
for i in (model.predict(pad_encoded)[0]).argsort()[-3:][::-1]:
  pred_word = tokenizer.index_word[i]
  print("Next word suggestion:",pred_word)

In [ ]:
# type( model.predict(pad_encoded)[0].argsort( )[-3:][::-1] )